In [0]:
%sql

SELECT Year, Quarter, OriginAirportID, Origin, DestAirportID, Dest, RPCarrier, SUM(Passengers) AS Tot_Passengers
FROM workspace.airline_origin_dest_data.airline_data_table
GROUP BY Year, Quarter, RPCarrier, OriginAirportID, DestAirportID, Origin, Dest

In [0]:
%sql

SELECT Year, Quarter, OriginAirportID, Origin, DestAirportID, Dest, TkCarrier, SUM(Passengers) AS Tot_Passengers
FROM workspace.airline_origin_dest_data.airline_data_table
GROUP BY Year, Quarter, TkCarrier, OriginAirportID, DestAirportID, Origin, Dest

In [0]:
%sql
SELECT Origin, Dest, OpCarrier, RpCarrier, TkCarrier, SUM(Passengers) AS Tot_Passengers
FROM workspace.airline_origin_dest_data.airline_data_table
WHERE OpCarrier != TkCarrier OR OpCarrier != RPCarrier
GROUP BY Origin, Dest, OpCarrier, RPCarrier, TkCarrier

In [0]:
%sql

SELECT Year, Quarter, OriginAirportID, Origin, DestAirportID, Dest, OpCarrier, SUM(Passengers) AS Tot_Passengers
FROM workspace.airline_origin_dest_data.airline_data_table
GROUP BY Year, Quarter, OpCarrier, OriginAirportID, DestAirportID, Origin, Dest

In [0]:
%sql

SELECT Year, Quarter, OriginAirportID, Origin, DestAirportID, Dest, OpCarrier, SUM(Passengers) AS Tot_Passengers
FROM workspace.airline_origin_dest_data.airline_data_table
WHERE OpCarrier == TkCarrier AND OpCarrier == RPCarrier
GROUP BY Year, Quarter, OpCarrier, OriginAirportID, DestAirportID, Origin, Dest

In [0]:
%sql

SELECT any_value(ItinID), Year, Quarter, CONCAT(Origin, " -> ", Dest) AS Route, RpCarrier, SUM(Passengers) AS Tot_Passengers, OriginAirportID, Origin, DestAirportID, Dest
FROM workspace.airline_origin_dest_data.airline_data_table
GROUP BY Year, Quarter, RpCarrier, OriginAirportID, DestAirportID, Origin, Dest

In [0]:
%sql
SELECT * FROM workspace.airline_origin_dest_data.airline_data_table
WHERE ItinID == "20251508"

In [0]:
%sql

SELECT * FROM workspace.airline_origin_dest_data.airline_data_table
WHERE AirportGroup == "PHL:RDU:CLT"

In [0]:
%sql

SELECT any_value(ItinID), Year, Quarter, split(AirportGroup, ":") AS legs, RpCarrier, SUM(Passengers) AS Tot_Passengers, OriginAirportID, Origin, DestAirportID, Dest
FROM workspace.airline_origin_dest_data.airline_data_table
GROUP BY Year, Quarter, RpCarrier, OriginAirportID, DestAirportID, Origin, Dest, legs

In [0]:
%sql
SELECT any_value(ItinID), Year, Quarter, split(AirportGroup, ':') AS legs, RpCarrier, SUM(Passengers) AS Tot_Passengers
FROM workspace.airline_origin_dest_data.airline_data_table
GROUP BY Year, Quarter, RpCarrier, legs

In [0]:
%sql
WITH exploded AS (
    SELECT
        ItinID, Year, Quarter, RpCarrier, Passengers,
        pos,
        value AS ap
    FROM workspace.airline_origin_dest_data.airline_data_table
    LATERAL VIEW posexplode(split(AirportGroup, ':')) t AS pos, value
)

SELECT
    e1.itinid,
    e1.year, e1.quarter, e1.rpcarrier,
    e1.passengers,
    e1.ap AS origin_airport,
    e2.ap AS dest_airport,
    e1.pos AS leg_number
FROM exploded e1
JOIN exploded e2
    ON e1.itinid = e2.itinid
   AND e1.pos + 1 = e2.pos
ORDER BY e1.itinid, leg_number;

In [0]:
%sql
WITH exploded AS (
    SELECT
        ItinID, Year, Quarter, RpCarrier, Passengers,
        pos,
        value AS ap
    FROM workspace.airline_origin_dest_data.airline_data_table
    LATERAL VIEW posexplode(split(AirportGroup, ':')) t AS pos, value
)

SELECT
    -- any_value(e1.itinid),
    e1.year, e1.quarter, e1.rpcarrier,
    -- e1.ap AS origin_airport,
    -- e2.ap AS dest_airport,
    -- e1.pos AS leg_number,
    CONCAT(e1.ap, ' -> ', e2.ap) AS route,
    SUM(e1.passengers)
FROM exploded e1
JOIN exploded e2
    ON e1.itinid = e2.itinid
   AND e1.pos + 1 = e2.pos
GROUP BY e1.year, e1.quarter, e1.rpcarrier, route
ORDER BY e1.rpcarrier, route